In [74]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
import openai
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [75]:
load_dotenv()

True

In [124]:
loader = TextLoader("../data/AC:DC.txt")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
db = Chroma.from_documents(documents, embedding, persist_directory="./new_chroma_db")

In [125]:
len(documents)

99

In [126]:
retriever = db.as_retriever()

acdc_retrieval_function = {
    "name": "get_acdc_information",
    "description": "Retrieve information about AC/DC from stored documents. Takes a 'query' string and returns a string containing the response of the retriever.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The Query the user is asking about AC/DC, it can be a simple question or a complex one."
            },
        },
        "required": ["query"]
    }
}

def get_acdc_information(query: str) -> str:
    """
    Given a query about AC/DC, retrieve relevant information using Chroma retriever.
    """
    docs = retriever.invoke(query)
    if not docs:
        return "No information found related to your query."
    return "\n\n".join([doc.page_content for doc in docs])

In [127]:
from google.genai.types import GenerateContentConfig
 
# Generation Config
config = GenerateContentConfig(
    system_instruction="You are a helpful assistant that use tools to access and retrieve information about AC/DC from a Vector Store database retriever.", 
    tools=[{"function_declarations": [acdc_retrieval_function]}], # define the functions that the LLM can use
)

In [128]:
client = openai.OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    default_headers={
        "HTTP-Referer": "your-site.com",
        "X-Title": "ACDC QA Bot"
    }
)


llm = ChatOpenAI(
    model="google/gemini-2.0-flash-001",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "your-site.com",
        "X-Title": "ACDC QA Bot"
    },
    temperature=0.2,
)

In [129]:
from langchain_core.messages import HumanMessage

messages = [
    {"role": "system", "content": "You are a helpful assistant that retrieves information about AC/DC."},
    {"role": "user", "content": "When was AC/DC founded?"}
]

response = client.chat.completions.create(
    model="google/gemini-2.0-flash-001",
    messages=messages,
    tools=[{"type": "function", "function": acdc_retrieval_function}],
    tool_choice="auto",
)

In [130]:
print(response)

ChatCompletion(id='gen-1745674202-OZCuDVy7FN5imkKMfdNH', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_get_acdc_information', function=Function(arguments='{"query":"When was AC/DC founded?"}', name='get_acdc_information'), type='function', index=0)], reasoning=None), native_finish_reason='STOP')], created=1745674202, model='google/gemini-2.0-flash-001', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=77, total_tokens=91, completion_tokens_details=None, prompt_tokens_details=None), provider='Google AI Studio')


In [131]:
import json


# Tool registry
functions = {
    "get_acdc_information": get_acdc_information
}

# Call function helper
def call_function(function_name, **kwargs):
    return functions[function_name](**kwargs)

# Agentic loop
def function_call_loop(prompt: str):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that retrieves information about AC/DC."},
        {"role": "user", "content": prompt}
    ]
    
    response = client.chat.completions.create(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        tools=[{"type": "function", "function": acdc_retrieval_function}],
        tool_choice="auto",
    )
    
    choice = response.choices[0]
    
    if choice.message.tool_calls:
        # Function tool call detected
        tool_call = choice.message.tool_calls[0]
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"🔧 Calling tool: {tool_name} with args: {arguments}")
        
        tool_result = call_function(tool_name, **arguments)
        
        # Now inform the LLM about the tool result
        tool_result_message = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_name,
            "content": json.dumps({"result": tool_result})
        }
        
        # Continue conversation
        followup_messages = [
            *messages,  # original system + user messages
            {
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "function": {
                            "name": tool_name,
                            "arguments": json.dumps(arguments)
                        },
                        "type": "function"
                    }
                ]
            },
            tool_result_message
        ]
        
        final_response = client.chat.completions.create(
            model="google/gemini-2.0-flash-001",
            messages=followup_messages,
        )
        
        final_text = final_response.choices[0].message.content
        return final_text
    
    elif choice.message.content:
        # Normal LLM direct response
        return choice.message.content
    
    else:
        return "⚠️ No valid response from model."

# Example usage
output = function_call_loop("I really like hard rock music, but i don't know many bands, what song would you recommend i listen to that is really popular")
print(output)


🔧 Calling tool: get_acdc_information with args: {'query': 'recommend a popular AC/DC song'}
I would recommend listening to "You Shook Me All Night Long". It's one of their most popular songs and a hard rock classic!



In [132]:
llm = ChatOpenAI(
    model="google/gemini-2.0-flash-001",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE"),
    default_headers={"HTTP-Referer": "your-site.com", "X-Title": "ACDC QA Bot"},
    temperature=0.2,
)

In [133]:
llm.invoke('hi there').content

'Hi! How can I help you today?\n'

In [134]:
from langchain.tools import tool

@tool
def get_acdc_information_tool(query: str) -> dict:
    """
    Given a query about AC/DC, retrieve relevant information using Chroma retriever.
    """
    docs = retriever.invoke(query)
    if not docs:
        return {"answer": "No information found related to your query."}

    top_snippets = [doc.page_content for doc in docs]
    snippet_text = "\n\n".join(top_snippets)

    return {"context": snippet_text}

llm_with_tools = llm.bind_tools([get_acdc_information_tool])

In [135]:
messages = [
    (
        "system",
        "You are a helpful assistant that use tools to access and retrieve information about AC/DC band.",
    ),
    ("human", "Who were the founders of AC/DC?"),
]
 
# Call the LLM with the messages and tools
res = llm_with_tools.invoke(messages)
 
# Check if the LLM returned a function call
if res.tool_calls:
    print(res.tool_calls)

[{'name': 'get_acdc_information_tool', 'args': {'query': 'Who were the founders of AC/DC?'}, 'id': 'tool_0_get_acdc_information_tool', 'type': 'tool_call'}]


In [136]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
 
# Initialize the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that uses tools to retrieve information about AC/DC. You MUST use tool outputs to answer questions, even if the outputs are long. Do your best to find the answer in the provided information."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

 
# Create the agent and executor with out llm, tools and prompt
agent = create_tool_calling_agent(llm_with_tools, [get_acdc_information_tool],prompt)
agent_executor = AgentExecutor(agent=agent, tools=[get_acdc_information_tool], verbose=True)
 
# Run our query 
res = agent_executor.invoke({"input": "Who were the founders of AC/DC?"})
print(res["output"])



> Entering new AgentExecutor chain...

Invoking: `get_acdc_information_tool` with `{'query': 'Who founded AC/DC?'}`


{'context': 'AC/DC were founded by brothers Angus (lead guitar) and Malcolm Young (rhythm guitar), with Colin Burgess (drums), Larry Van Kriedt (bass guitar) and Dave Evans (lead vocals). They underwent several line-up changes before releasing their debut Australasian-only album, High Voltage (1975). Membership stabilised after the release of Let There Be Rock (1977), with the Young brothers, Phil Rudd on drums, Cliff Williams on bass guitar and Bon Scott on lead vocals. Seven months after the release of Highway to Hell (1979), Scott died of alcohol poisoning and English singer Brian Johnson was then recruited as their new frontman. Their first album with Johnson, Back in Black (1980), dedicated to Scott\'s memory, became the second best-selling album of all time. Their eighth studio album, For Those About to Rock (1981), was their first album to reach number one on t

In [137]:
res = agent_executor.invoke({"input": "Which would you say is AC/DC's most popular song, from what you know estimate"})
print(res["output"])




> Entering new AgentExecutor chain...

Invoking: `get_acdc_information_tool` with `{'query': 'AC/DC most popular song'}`


{'context': 'They sold over 1.3 million CDs in the US during 2007.[289] In 50 years of the band\'s career, they have sold over 200 million albums worldwide,[290] and 84 million in the US, according to the Recording Industry Association of America (RIAA), which AC/DC the fourth best-selling band in US history and the eighth best-selling artist, selling more albums than Elton John and Mariah Carey.[291] The RIAA also certified Back in Black as 27× Platinum, for 27 million in US sales, which made it the fourth best-selling album of all time in the US.[292]\n\nAC/DC are an Australian rock band formed in Sydney in 1973. Their music has been variously described as hard rock, blues rock and heavy metal, although the band calls it simply "rock and roll". They are cited as a formative influence on the new wave of British heavy metal bands. The band was inducted into the R

In [142]:
import shutil

# Path where Chroma DB is persisted
chroma_path = "./chroma_db"

# Delete the whole folder and everything inside it
shutil.rmtree(chroma_path, ignore_errors=True)

print(f"✅ Deleted {chroma_path}")


✅ Deleted ./chroma_db
